In [1]:
###########################################
# IMPORTS
###########################################
from simtk.openmm import app
from simtk import unit, openmm
import numpy as np
import os
import random
from nose.tools import nottest
from pkg_resources import resource_filename


from perses.annihilation.relative import HybridTopologyFactory
from perses.rjmc.geometry import FFAllAngleGeometryEngine
from perses.tests import utils
from openmmtools.states import SamplerState
import openmmtools.mcmc as mcmc
import openmmtools.cache as cache
from unittest import skipIf

import pymbar.timeseries as timeseries

import pymbar

running_on_github_actions = os.environ.get('GITHUB_ACTIONS', None) == 'true'

try:
    cache.global_context_cache.platform = openmm.Platform.getPlatformByName("Reference")
except Exception:
    cache.global_context_cache.platform = openmm.Platform.getPlatformByName("Reference")

#############################################
# CONSTANTS
#############################################
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
CARBON_MASS = 12.01
ENERGY_THRESHOLD = 1e-1
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("Reference")
aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']


INFO:numexpr.utils:Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [23]:
def RepartitionedHybridTopologyFactory_energies(topology, chain, system, positions, system_generator):
    """
    Test whether the difference in the nonalchemical zero and alchemical zero states is the forward valence energy.  Also test for the one states.
    Note that two RepartitionedHybridTopologyFactorys need to be generated (one for each endstate) because the energies need to be validated separately for each endstate.
    """

    from perses.rjmc.topology_proposal import PointMutationEngine
    from openmmtools.integrators import LangevinIntegrator
    from perses.annihilation.relative import RepartitionedHybridTopologyFactory
    from perses.tests.utils import validate_endstate_energies

    ENERGY_THRESHOLD = 1e-6

    for res in topology.residues():
        if res.id == '2':
            wt_res = res.name
    aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
    mutant = random.choice(aminos_updated)
#     mutant = 'GLU'
    print(f'Making mutation {wt_res}->{mutant}')

    # Create point mutation engine to mutate residue at id 2 to random amino acid
    point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                                system_generator=system_generator,
                                                chain_id=chain,
                                                max_point_mutants=1,
                                                residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                                allowed_mutations=[('2', mutant)],
                                                aggregate=True)  # always allow aggregation

    # Create topology proposal
    topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

    # Create geometry engine
    from perses.rjmc.geometry import FFAllAngleGeometryEngine
    geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                               use_sterics=False,
                                               n_bond_divisions=100,
                                               n_angle_divisions=180,
                                               n_torsion_divisions=360,
                                               verbose=True,
                                               storage=None,
                                               bond_softening_constant=1.0,
                                               angle_softening_constant=1.0,
                                               neglect_angles=False,
                                               use_14_nonbondeds=True)
    
    # Create geometry proposal
    for i in range(5):
        new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                                       validate_energy_bookkeeping=True)
        logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                                    validate_energy_bookkeeping=True)

        # Check potential energy
        platform = openmm.Platform.getPlatformByName('Reference')
        integrator = LangevinIntegrator(temperature=temperature)
        context = openmm.Context(topology_proposal.new_system, integrator, platform)
        context.setPeriodicBoxVectors(*topology_proposal.new_system.getDefaultPeriodicBoxVectors())
        context.setPositions(new_positions)
        context.setVelocitiesToTemperature(temperature)
        potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
    #     openmm.LocalEnergyMinimizer.minimize(context)

        print("\t\t\t\t POTENTIAL ENERGY of proposal: ", potential_energy)
        if potential_energy < 1e8 * unit.kilojoule_per_mole:
            print("potential energy is fine! proceed")
            break
        else:
            print("potential energy is not fine! generate a new proposal")
            if i == 4:
                raise Exception("Was not able to generate a decent proposal after 5 iterations")
    
    print("broke out of geom proposal loop")
    
    if not topology_proposal.unique_new_atoms:
        assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
        assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
        vacuum_added_valence_energy = 0.0
    else:
        added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

    if not topology_proposal.unique_old_atoms:
        assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
        assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
        subtracted_valence_energy = 0.0
    else:
        subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential

    print("\t\t\t ADDED VALENCE: ", added_valence_energy)
    print("\t\t\t SUBTRACTED VALENCE: ", subtracted_valence_energy)
    print("\t\t\t forward_final_context_reduced_potential: ", geometry_engine.forward_final_context_reduced_potential)
    print("\t\t\t forward_atoms_with_positions_reduced_potential: ", geometry_engine.forward_atoms_with_positions_reduced_potential)
    
        
    # Generate repartitioned htf at lambda = 0
    zero_htf = RepartitionedHybridTopologyFactory(topology_proposal=topology_proposal,
                          current_positions=positions,
                          new_positions=new_positions,
                          endstate=0)

    # Compute error at lambda = 0 endstate
    zero_state_error, _ = validate_endstate_energies(zero_htf._topology_proposal,
                                                     zero_htf,
                                                     added_valence_energy,
                                                     subtracted_valence_energy,
                                                     ENERGY_THRESHOLD=ENERGY_THRESHOLD,
                                                     platform=openmm.Platform.getPlatformByName('Reference'),
                                                     repartitioned_endstate=0)
    # Generate repartitioned htf at lambda = 1
    one_htf = RepartitionedHybridTopologyFactory(topology_proposal=topology_proposal,
                                                  current_positions=positions,
                                                  new_positions=new_positions,
                                                  endstate=1)

    # Compute error at lambda = 1 endstate
    _, one_state_error = validate_endstate_energies(one_htf._topology_proposal,
                                                     one_htf,
                                                     added_valence_energy,
                                                     subtracted_valence_energy,
                                                     ENERGY_THRESHOLD=ENERGY_THRESHOLD,
                                                     platform=openmm.Platform.getPlatformByName('Reference'),
                                                     repartitioned_endstate=1)

    # Check that endstate errors are below threshold
    assert abs(zero_state_error) < ENERGY_THRESHOLD, f"The zero state alchemical and nonalchemical energy absolute difference {abs(zero_state_error)} is greater than the threshold of {ENERGY_THRESHOLD}."
    assert abs(one_state_error) < ENERGY_THRESHOLD, f"The one state alchemical and nonalchemical energy absolute difference {abs(one_state_error)} is greater than the threshold of {ENERGY_THRESHOLD}."

    print(f"Abs difference in zero alchemical vs nonalchemical systems: {abs(zero_state_error)}")
    print(f"Abs difference in one alchemical vs nonalchemical systems: {abs(one_state_error)}")


In [24]:
def test_RepartitionedHybridTopologyFactory_energies():
    """
    Test whether the difference in the nonalchemical zero and alchemical zero states is the forward valence energy.  Also test for the one states.
    """

    from perses.tests.test_topology_proposal import generate_atp
    from openmmforcefields.generators import SystemGenerator

    # Test alanine dipeptide in vacuum
    atp, system_generator = generate_atp('vacuum')
    RepartitionedHybridTopologyFactory_energies(atp.topology, '1', atp.system, atp.positions, system_generator)

    # Test alanine dipeptide in solvent
    atp, system_generator = generate_atp('solvent')
    RepartitionedHybridTopologyFactory_energies(atp.topology, '1', atp.system, atp.positions, system_generator)

    # Test 8-mer peptide in vacuum
    peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
    pdb = app.PDBFile(peptide_filename)
    forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
    barostat = None
    system_generator = SystemGenerator(forcefields=forcefield_files,
                                       barostat=barostat,
                                       forcefield_kwargs={'removeCMMotion': False,
                                                          'ewaldErrorTolerance': 0.00025,
                                                          'constraints': app.HBonds,
                                                          'hydrogenMass': 4 * unit.amus},
                                       periodic_forcefield_kwargs=None,
                                       small_molecule_forcefield='gaff-2.11',
                                       nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                       molecules=None,
                                       cache=None)
    system = system_generator.create_system(pdb.topology)
    positions = unit.quantity.Quantity(
                    value=np.array([list(atom_pos) for atom_pos in pdb.positions.value_in_unit_system(unit.md_unit_system)]),
                    unit=unit.nanometers)
    RepartitionedHybridTopologyFactory_energies(pdb.topology, 'C', system, positions, system_generator)


In [26]:
for i in range(10):
    test_RepartitionedHybridTopologyFactory_energies()

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{12: 8}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {10: 9}, {9: 9}, {15: 8}, {14: 8}, {13: 8}, {9: 7}, {11: 8}, {10: 8}, {9: 8}, {15: 7}, {14: 7}, {13: 7}, {12: 7}, {11: 7}, {10: 7}]
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{15: 9}
INFO:proposal_generator:{14: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{15: 8}
INFO:proposal_generator:{14: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{

Making mutation ALA->VAL
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:	reduced angle potential = 0.13308416651072802.
INFO:geometry:	reduced angle potential = 0.43866177283799707.
INFO:geometry:	reduced angle potential = 0.10550914124373352.
INFO:geometry:	reduced angle potential = 0.7012295147245629.
INFO:geometry:	reduced angle potential = 0.00018984355655466805.
INFO:geometry:	reduced angle potential = 0.36522839559000564.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 48 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defin

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 3.7432562392201634), ('CustomAngleForce', 11.376163490513395), ('CustomTorsionForce', 11.845922717078073), ('CustomBondForce', -62.34490689656943)]
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducti

INFO:geometry:	reduced angle potential = 7.39096069988752e-11.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 9.116481158706168
INFO:geometry:total reduced energy added from growth system: 20.67298346458353
INFO:geometry:final reduced energy 29.789464742816335
INFO:geometry:sum of energies: 29.7894646232897
INFO:geometry:magnitude 

...

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 2.4614261578130217), ('CustomAngleForce', 240.93245820532823), ('CustomTorsionForce', 22.335684665241164), ('CustomBondForce', 396.4412102807236)]
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 61 bond forces in the no-nonbonded final system
INFO:geometry:	there are 218 angle forces in the no-nonbonded final system
INFO:geometry:	there are 399 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 168.3666171145502
INFO:geometry:total reduced energy added from growth system: 19.113357542933382
INFO:geometry:final reduced energy 187.47997478601175
INFO:geometry:sum of energies: 187.47997465748358
INFO:geometry:magnitude of difference in the energies: 1.2852817476982636e-07
INFO:geometry:Final logp_proposal: -30500.941354740964
INFO:relative:*** Generating RepartitionedHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'Peri

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.04562514569649219), ('CustomAngleForce', 0.11963265647716706), ('CustomTorsionForce', 7.457414093494336), ('CustomBondForce', 11.490685647265389)]
				 POTENTIAL ENERGY of proposal:  13970.518666793196 kJ/mol
potential energy is fine! proceed
broke out of geom proposal loop
			 ADDED VALENCE:  662.1707781493733
			 SUBTRACTED VALENCE:  19.113357671461557
			 forward_final_context_reduced_potential:  830.5373952639235
			 forward_

INFO:relative:*** Generating RepartitionedHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system exceptions dict...
INFO:relative:H

conducting subsequent work with the following platform: CPU
			HarmonicBondForce: 7.300097488950959
			HarmonicAngleForce: 12.445683175437441
			PeriodicTorsionForce: 156.24350834582967
			NonbondedForce: -174.2619522021362
			AndersenThermostat: 0.0
conducting subsequent work with the following platform: CPU
			HarmonicBondForce: 9.76152364676398
			HarmonicAngleForce: 253.37814138076573
			PeriodicTorsionForce: 178.5791930110708
			NonbondedForce: 222.17925691885446
			AndersenThermostat: 0.0
conducting subsequent work with the following platform: CPU
			HarmonicBondForce: 9.715898501067489
			HarmonicAngleForce: 253.2585087242886
			PeriodicTorsionForce: 171.12177891757648
			NonbondedForce: 5166.813957156308
			AndersenThermostat: 0.0
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
			HarmonicBondForce: 7.300097488950959
			HarmonicAngleForce: 12.445683175437441
			PeriodicTorsionForce: 156.24350834582967